In [3]:
from numpy.typing import ArrayLike
import numpy as np
import random

In [53]:
def reverse_dir(dir):
    if dir == 'l':
        return 'r'
    if dir == 'r':
        return 'l'
    if dir == 'u':
        return 'd'
    if dir == 'd':
        return 'u'



class state_generator:
    dirs = ['l', 'r', 'u', 'd']
    ind_generators = {
        'l' : lambda ind : (ind[0], ind[1] - 1),
        'u' : lambda ind : (ind[0] - 1, ind[1]),
        'r' : lambda ind : (ind[0], ind[1] + 1),
        'd' : lambda ind : (ind[0] + 1, ind[1])
    }


    def __init__(self, n : int=4) -> None:
        self.n = n

    
    def random_state(self):                     # генерирует абсолютно случайное состояние (не факт что валидное)
        lst = np.arange(1, self.n**2 + 1)
        lst[-1] = 0
        np.random.shuffle(lst)
        return lst.reshape(self.n, self.n)
    

    def random_valid_state(self):               # генерирует случайное валидное состояние (путем преобразования массива решений)
        state = self.solution()
        dirs_to_solve = []                      # сюда мы засунем совершенную перестановку
        n = random.randint(5, 10)
        prev_dir = None

        for i in range(n):
            tmp_states = self.possible_states(state)
            tmp_dirs = [i for i in list(tmp_states.keys()) if i != reverse_dir(prev_dir)]
            dir = random.choice(tmp_dirs)
            dirs_to_solve.append(dir)
            state = tmp_states[dir]
            prev_dir = dir
        return state, dirs_to_solve


    def solution(self):                             # создает решение заданного размера
        lst = np.arange(1, self.n**2 + 1)
        lst[-1] = 0
        return lst.reshape(self.n, self.n)
    
    
    def _do_swap(self, state_arr:ArrayLike, dir:str, ind:ArrayLike):    #swap нулевого элемента в сторону dir
        new_state = np.copy(state_arr)
        if dir not in state_generator.dirs:
            raise Exception(f"unknown direction, dir must be in {state_generator.dirs}")
        if not self._is_valid_dir(dir, ind):
            return None
        
        new_ind = state_generator.ind_generators[dir](ind)
        new_state[ind], new_state[new_ind] = new_state[new_ind], new_state[ind]
        return new_state
    

    def _is_valid_dir(self, dir, ind):                                    # возвращает, допустим ли свап в направление dir
        if  (dir == 'l' and ind[1] == 0) or (dir == 'r' and ind[1] == self.n - 1) or \
            (dir == 'u' and ind[0] == 0) or (dir == 'd' and ind[0] == self.n - 1):
            return False
        return True

    def possible_states(self, state_arr):                                 # возвращает словарь {dir : arr} с допустимыми свапами
        ind = np.where(state_arr == 0)
        dict_states = {}
        for dir in state_generator.dirs:
            new_state = self._do_swap(state_arr, dir, ind)
            if new_state is not None:
                dict_states[dir] = new_state
        return dict_states



class SolvabilityTester:
    def __init__(self, n=4, gen=None) -> None:
        self.gen = gen
        self.n = n
        if self.gen is None:
            self.gen = state_generator(n)        
        self.solved_snake_inv = self.get_solved_snake_inv()


    def get_solved_snake_inv(self):
        solved_state = self.gen.solution()
        solved_snake_arr = self.generate_snake_perm(solved_state)
        return self.count_inversions(solved_snake_arr)

    def get_snake_inv(self, arr):
        return self.count_inversions(self.generate_snake_perm(arr))
        
    def is_solvable(self, arr):
        # print(self.solved_snake_inv % 2, self.get_snake_inv(arr) % 2)
        return self.solved_snake_inv % 2 == self.get_snake_inv(arr) % 2

    def generate_snake_perm(self, state):
        snake = []
        for i in range(self.n):
            if i % 2 == 0:
                for j in range(self.n):
                    if state[i][j] != 0:
                        snake.append(state[i][j])
            else:
                for j in range(self.n - 1, -1, -1):
                    if state[i][j] != 0:
                        snake.append(state[i][j])
        return snake

    def count_inversions(self, arr):
        inv = 0
        for i in range(len(arr)):
            for j in range(i):
                if arr[j] > arr[i]:
                    inv += 1
        return inv


        

In [54]:
class Solution:
    def __init__(self, n : int=4):
        self.n = n
        self.generator = state_generator(n)
    
    def get_solution(self, arr:ArrayLike):
        if np.shape(arr) != (self.n, self.n):
            raise Exception(f"Shape must be {(self.n, self.n)}")
        raise NotImplementedError

In [55]:
gen = state_generator(4)
s = SolvabilityTester(4, gen)

In [57]:
for i in range(100):
    state, seq = gen.random_valid_state()
    if not s.is_solvable(state):
        print("False negative")
    if s.is_solvable(np.rot90(state)):
        print("False positive")
